## MERLIN MODELS

Recommender Systems (RecSys) attempt to accurately predict users' preferences by supporting users in the process of finding and selecting products (items) from a given catalog, thus enhance their engagement and overall satisfaction with online services. Several traditional (e.g. collaborative filtering) and Deep Learning-based RecSys models have been built and used in industry within the last decade. Recently, deep learning based recommender systems have become the norm for large scale industry applications. In contrast to the other domains, DL recommender models aren’t able to effectively leverage the parallel compute and high memory bandwidth that GPUs have to offer. To address the challenges in preprocessing, training and deploying recommender systems at scale, NVIDIA developed an open source framework, called [Merlin](https://github.com/NVIDIA-Merlin). One of the libraries of the Merlin framework is Merlin Models that is being developed to provide all the necessary tools to design an end-to-end RecSys pipeline, offering flexibility at each stage: multiple inputs processing/representation modules, different layers for designing the model’s architecture, different prediction heads, loss functions, different negative sampling techniques etc. 

The goal of the `Merlin Models` library is make it easy for users in industry or academia to train and deploy recommender models with best practices baked into the library. This will let users in industry easily train standard models against their own dataset, getting high performance GPU accelerated models into production. This will also let researchers to build custom models by incorporating standard components of deep learning recommender models, and then benchmark their new models on example offline datasets.

### Core features

- Support for diverse input types (e.g. categorical and continuous), architectures (e.g. two-tower or sequential) or tasks (e.g. binary, multi-class classification, multi-task)
- Flexible building blocks to define a broad range of models with various prediction tasks, loss functions and negative sampling techniques 
- Flexible APIs targeted to both production and research
- Unified API enables users to create models in TensorFlow or PyTorch
- GPU-optimized data-loading, model training and serving
- Deep integration with NVTabular for ETL and model serving
- Multi-task learning is a first-class citizen
- State-of-the-art negative sampling techniques

### Learning objectives

In this introductory notebook we aim at

- introducing the building blocks of Merlin Models library
- showing how seamlessly integrate NVTabular and Merlin Models libraries
- training DL-based recommender models with optimized Tensorflow data loaders with only a few lines of code for `BinaryClassification` task. 

## Import Libraries

In [1]:
import os
import glob

from nvtabular.loader.tensorflow import KerasSequenceLoader

import merlin_models.tf as ml
from merlin_standard_lib import Schema, Tag

2022-01-12 15:39:22.450339: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 16254 MB memory:  -> device: 0, name: Quadro GV100, pci bus id: 0000:15:00.0, compute capability: 7.0


## ETL with NVTabular

We use the [MovieLens25M](https://grouplens.org/datasets/movielens/25m/) dataset which is a popular dataset for recommender systems and is used in academic publications. In order to download and convert the raw dataset, run `01-Download-Convert.ipynb` notebook first. Here, we will use parquet files processed using [NVTabular](https://github.com/NVIDIA-Merlin/NVTabular) library. Run the `02-ETL-with-NVTabular.ipynb` notebook to generate the input features and processed parquet files. The ETL notebook explains all the preprocessing and feature engineering steps in details.

In [2]:
# disable INFO and DEBUG logging everywhere
import logging
logging.disable(logging.WARNING)

Avoid Numba low occupancy warnings

In [3]:
from numba import config
config.CUDA_LOW_OCCUPANCY_WARNINGS = 0

We define our base input directory, containing the data.

In [4]:
INPUT_DATA_DIR = os.environ.get(
    "INPUT_DATA_DIR", os.path.expanduser("/workspace/data/movielens/")
)

### Schema Object

Merlin Models library relies on a schema object that takes the input features as input and automatically builds all necessary layers to represent, normalize and aggregate input features. As you can see below, `schema.pbtxt` is a protobuf text file contains features metadata, including statistics about features such as cardinality, min and max values and also tags based on their characteristics and dtypes (e.g., categorical, continuous, list, item_id). We can tag our target column and even add the prediction task such as binary, regression or multiclass as tags for the target column in the `schema.pbtxt` file. The Schema provides a standard representation for metadata that is useful when training machine learning or deep learning models.

The metadata information loaded from Schema and their tags are used to automatically set the parameters of Merlin models. Certain modules have a `from_schema()` method to instantiate their parameters and layers from protobuf text file respectively.

We have already generated our `schema.pbtxt` file in the previous notebook using `NVTabular`. Now we start with reading this schema file to create a schema object.

In [5]:
from merlin_standard_lib import Schema
SCHEMA_PATH = "/workspace/data/movielens/train/schema.pbtxt"
schema = Schema().from_proto_text(SCHEMA_PATH)
!head -30 $SCHEMA_PATH

feature {
  name: "movieId"
  type: INT
  int_domain {
    name: "movieId"
    min: 0
    max: 56690
    is_categorical: true
  }
  annotation {
    tag: "item_id"
    tag: "categorical"
    tag: "item"
    extra_metadata {
      type_url: "type.googleapis.com/google.protobuf.Struct"
      value: "\n\021\n\013num_buckets\022\002\010\000\n\033\n\016freq_threshold\022\t\021\000\000\000\000\000\000\000\000\n\025\n\010max_size\022\t\021\000\000\000\000\000\000\000\000\n\030\n\013start_index\022\t\021\000\000\000\000\000\000\000\000\n2\n\010cat_path\022&\032$.//categories/unique.movieId.parquet\nG\n\017embedding_sizes\0224*2\n\030\n\013cardinality\022\t\021\000\000\000\000@\256\353@\n\026\n\tdimension\022\t\021\000\000\000\000\000\000\200@"
    }
  }
}
feature {
  name: "userId"
  type: INT
  int_domain {
    name: "userId"
    min: 0
    max: 162542
    is_categorical: true
  }
  annotation {
    tag: "user_id"


## Building an MLP with Merlin Models

Let's start with building an MLP model to train a binary classification task using Merlin Models library. As a starting point, we are going to use only `['movieId', 'userId', 'genres']` input features. Note that our target column is `rating_binary` created in the `02-ETL-with-NVTabular` notebook as a binary target column.

We can easily remove the features we do not want to use as input to the model with `remove_by_name` property of the schema object, and create a new schema.

In [6]:
schema = schema.remove_by_name(['rating', 'title', 'TE_movieId_rating', 'userId_count'])

In [7]:
schema.column_names

['movieId', 'userId', 'genres', 'rating_binary']

Below we build a TF MLP model using three main blocks:
- [InputBlock](https://github.com/NVIDIA-Merlin/models/blob/main/merlin_models/tf/block/inputs.py) is the entry block of the model that accepts `schema` object and other arguments like `aggregation`, `continuous projection`, `embedding options`. This function creates continuous and embedding layers, and connects them via [ParallelBlock](https://github.com/NVIDIA-Merlin/models/blob/main/merlin_models/tf/core.py#L1184). If `aggregation` argument is not set, it returns a dictionary of multiple tensor each corresponds to an input feature, otherwise it merges the tensors into one using the aggregation method.
- [ParallelBlock](https://github.com/NVIDIA-Merlin/models/blob/main/merlin_models/tf/core.py#L1184) take a list of layers as input, stacks them in parallel, and then  outputs a dictionary of tensors.
- [MLPBlock](https://github.com/NVIDIA-Merlin/models/blob/main/merlin_models/tf/block/mlp.py) is to define a Multi-layer perceptron block, where each dimension is used to create a fully connected Dense layer. In addition to `dimension` argument we can also feed `activation`, `use_bais`, `dropout`, etc. arguments to the `MLPBlock` function.
- [BinaryClassificationTask](https://github.com/NVIDIA-Merlin/models/blob/main/merlin_models/tf/prediction/classification.py#L30) supports the binary prediction task. Merlin Models library also supports other predictions tasks, like next-item prediction and regression.

In [8]:
# default emb_dim is 64
model = ml.Model(
    ml.InputBlock(schema), 
    ml.MLPBlock([64, 32]),
    ml.BinaryClassificationTask("rating_binary")
)

As you can notice we start with `InputBlock`, then create an `MLP block`, and then finally, we connect our `MLP block` to the BinaryClassificationTask head to be able to do binary classification, and create our model class. Basically, this is similar to a linear stacking of layers into a tf.keras.Model as done with tf.keras.Sequential.

### Define Data Loader

We're ready to start training. We'll use the NVTabular `KerasSequenceLoader` for reading chunks of parquet files. `KerasSequenceLoader` manages shuffling by loading in chunks of data from different parts of the full dataset, concatenating them and then shuffling, then iterating through this super-chunk sequentially in batches. The number of "parts" of the dataset that get sample, or "partitions", is controlled by the `parts_per_chunk` kwarg, while the size of each one of these parts is controlled by the `buffer_size` kwarg, which refers to a fraction of available GPU memory (you can read more about it [here](https://nvidia-merlin.github.io/NVTabular/main/training/tensorflow.html) and [here](https://nvidia-merlin.github.io/NVTabular/main/api/tensorflow_dataloader.html?highlight=kerassequence#nvtabular.loader.tensorflow.KerasSequenceLoader)). Using more chunks leads to better randomness, especially at the epoch level where physically disparate samples can be brought into the same batch, but can impact throughput if you use too many. In any case, the speed of the parquet reader makes feasible buffer sizes much larger.

Note that `genres` column is a multi-hot column and it is fed to dataloader as a sparse tensor and then it is converted to dense represantation. Based on our analysis, genres column has max 10 sequence of entries. So we will set the sequence length for the multi-hot columns as 10 in the `sparse_feature_max` dictionary below.

Note that we do not have continuous columns so we only feed categorical columns to the model.

In [9]:
# Define categorical and continuous columns
x_cat_names, x_cont_names = ['userId', 'movieId', 'genres'], []

# dictionary representing max sequence length for each column
sparse_features_max = {'genres': 10}

def get_dataloader(paths_or_dataset, batch_size=4096):
    dataloader = KerasSequenceLoader(
        paths_or_dataset,
        batch_size=batch_size,
        label_names=['rating_binary'],
        cat_names=x_cat_names,
        cont_names=x_cont_names,
        sparse_names=list(sparse_features_max.keys()),
        sparse_max=sparse_features_max,
        sparse_as_dense=True,
    )
    return dataloader.map(lambda X, y: (X, tf.reshape(y, (-1,))))

Define the parquet file paths for model training.

In [10]:
OUTPUT_DIR = os.environ.get("OUTPUT_DIR", "/workspace/data/movielens/")
train_paths = glob.glob(os.path.join(OUTPUT_DIR, "train/*.parquet"))
eval_paths = glob.glob(os.path.join(OUTPUT_DIR, "valid/*.parquet"))

In [11]:
import tensorflow as tf
model.compile(optimizer="adam", run_eagerly=False)

In order to train and evaluate our model we are using `.fit()` and `.evaluate()` methods as done in `tf.keras`.

In [12]:
print('*'*20)
print("Launch training")
print('*'*20 + '\n')
train_loader = get_dataloader(train_paths) 
losses = model.fit(train_loader, epochs=1)
model.reset_metrics()

# Evaluate
print('*'*20 + '\n')
print("Start evaluation")
print('*'*20 + '\n')
eval_loader = get_dataloader(eval_paths) 
eval_metrics = model.evaluate(eval_loader, return_dict=True)

********************
Launch training
********************



2022-01-12 15:41:35.065217: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-01-12 15:41:37.353543: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: annotated name 'output' can't be nonlocal (tmpfdeqazw6.py, line 36)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
4883/4883 [==============================] - 191s 12ms/step - rating_binary/binary_classification_task/precision: 0.7620 - rating_binary/binary_classification_task/recall: 0.8615 - rating_binary/binary_classification_task/binary_accuracy: 0.7452 - rating_binary/binary_classification_task/auc: 0.8024 - loss: 0.5136 - regularization_loss: 0.0000e+00 - total_loss: 0.5136
********************

Start evaluation
********************

1221/1221 [==============================] - 43s 8ms/step - rating_binary/binary_classification_task/precision: 0.7771 - rating_binary/binary_classification_task/recall: 0.8573 - rating_binary/binary_classification_task/binary_accuracy: 0.

#### Connect Method

Now we will build the same MLP model using `connect` method. Connect method cares about the sequential order, first argument is run first. The `connect` method is works as [SequentialBlock]() class which enables users to represent a sequence of Keras layers. It is a Keras Layer that can be used instead of `tf.keras.layers.Sequential` which is actually a Keras Model. 

In [13]:
model = ml.InputBlock(schema).connect(ml.MLPBlock([64, 32]), ml.BinaryClassificationTask("rating_binary"))

In [14]:
model.compile(optimizer="adam", run_eagerly=False)

In [15]:
print('*'*20)
print("Launch training")
print('*'*20 + '\n')
train_loader = get_dataloader(train_paths) 
losses = model.fit(train_loader, epochs=1)
model.reset_metrics()

# Evaluate
print('*'*20 + '\n')
print("Start evaluation")
print('*'*20 + '\n')
eval_loader = get_dataloader(eval_paths) 
eval_metrics = model.evaluate(eval_loader, return_dict=True)

********************
Launch training
********************

4883/4883 [==============================] - 191s 12ms/step - rating_binary/binary_classification_task/precision: 0.7616 - rating_binary/binary_classification_task/recall: 0.8608 - rating_binary/binary_classification_task/binary_accuracy: 0.7445 - rating_binary/binary_classification_task/auc: 0.8019 - loss: 0.5141 - regularization_loss: 0.0000e+00 - total_loss: 0.5141
********************

Start evaluation
********************

1221/1221 [==============================] - 42s 8ms/step - rating_binary/binary_classification_task/precision: 0.7720 - rating_binary/binary_classification_task/recall: 0.8670 - rating_binary/binary_classification_task/binary_accuracy: 0.7567 - rating_binary/binary_classification_task/auc: 0.8192 - loss: 0.4951 - regularization_loss: 0.0000e+00 - total_loss: 0.4951


Before moving to the next section we need to restart our kernel so that we can free up the GPU memory.

In [16]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

You can restart manually in case the restart is not sucessful as expected.

### ParallelBlock

Now, we will add continuous features and create a new model using `ParallelBlock` class. We will create one block (like a branch) for continuous features and another one for categrical features and we will connect them. Note that this architecture is different than the ones above.

In [17]:
import os
import glob

from nvtabular.loader.tensorflow import KerasSequenceLoader

import merlin_models.tf as ml
from merlin_standard_lib import Schema, Tag

In [18]:
SCHEMA_PATH = "/workspace/data/movielens/train/schema.pbtxt"
schema = Schema().from_proto_text(SCHEMA_PATH)

In [19]:
schema = schema.remove_by_name(['rating', 'title'])

In [20]:
schema.column_names

['movieId',
 'userId',
 'genres',
 'TE_movieId_rating',
 'userId_count',
 'rating_binary']

Note that  `TE_movieId_rating`,  and  `userId_count` are the continuous features.

In [21]:
con_schema = schema.select_by_tag(Tag.CONTINUOUS)
cat_schema = schema.select_by_tag(Tag.CATEGORICAL)

In [22]:
con_schema.column_names

['TE_movieId_rating', 'userId_count']

In [23]:
cat_schema.column_names

['movieId', 'userId', 'genres']

In [24]:
cont_block = ml.InputBlock(con_schema).connect(ml.MLPBlock([256, 64]))

cat_block = ml.InputBlock(cat_schema).connect(ml.MLPBlock([128, 64]))

body = ml.ParallelBlock({'continuous_block': cont_block, 'categorical_block': cat_block}, aggregation='concat')

model = body.connect(ml.BinaryClassificationTask("rating_binary"))

In [25]:
OUTPUT_DIR = os.environ.get("OUTPUT_DIR", "/workspace/data/movielens/")
train_paths = glob.glob(os.path.join(OUTPUT_DIR, "train/*.parquet"))
eval_paths = glob.glob(os.path.join(OUTPUT_DIR, "valid/*.parquet"))

Notice below that this time our `x_cont_names` list is not empty.

In [26]:
# Define categorical and continuous columns
x_cat_names, x_cont_names = ['userId', 'movieId', 'genres'], ['TE_movieId_rating', 'userId_count']

# dictionary representing max sequence length for each column
sparse_features_max = {'genres': 10}

def get_dataloader(paths_or_dataset, batch_size=4096):
    dataloader = KerasSequenceLoader(
        paths_or_dataset,
        batch_size=batch_size,
        label_names=['rating_binary'],
        cat_names=x_cat_names,
        cont_names=x_cont_names,
        sparse_names=list(sparse_features_max.keys()),
        sparse_max=sparse_features_max,
        sparse_as_dense=True,
    )
    return dataloader.map(lambda X, y: (X, tf.reshape(y, (-1,))))

In [27]:
import tensorflow as tf
model.compile(optimizer="adam", run_eagerly=False)

In [28]:
print('*'*20)
print("Launch training")
print('*'*20 + '\n')
train_loader = get_dataloader(train_paths) 
losses = model.fit(train_loader, epochs=1)
model.reset_metrics()

# Evaluate
print('*'*20 + '\n')
print("Start evaluation")
print('*'*20 + '\n')
eval_loader = get_dataloader(eval_paths) 
eval_metrics = model.evaluate(eval_loader, return_dict=True)

********************
Launch training
********************

4883/4883 [==============================] - 197s 13ms/step - rating_binary/binary_classification_task/precision: 0.7624 - rating_binary/binary_classification_task/recall: 0.8621 - rating_binary/binary_classification_task/binary_accuracy: 0.7458 - rating_binary/binary_classification_task/auc: 0.8032 - loss: 0.5127 - regularization_loss: 0.0000e+00 - total_loss: 0.5127
********************

Start evaluation
********************

1221/1221 [==============================] - 44s 9ms/step - rating_binary/binary_classification_task/precision: 0.7740 - rating_binary/binary_classification_task/recall: 0.8639 - rating_binary/binary_classification_task/binary_accuracy: 0.7571 - rating_binary/binary_classification_task/auc: 0.8199 - loss: 0.4943 - regularization_loss: 0.0000e+00 - total_loss: 0.4943


Like that we can easily build `BinaryClasificationTask` models with Merlin Models library in couple of lines. For more advanced ranking examples, please visit `ranking` folder and explore the other example notebooks.